In [1]:
!pip install streamlit
!pip install pyngrok
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not curre

In [2]:
from google.colab import files

# Upload the ZIP file
uploaded = files.upload()

Saving model_d (2).zip to model_d (2).zip


In [3]:
import zipfile
import os

# Get the name of the uploaded ZIP file
zip_file_name = list(uploaded.keys())[0]

# Extract the ZIP file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("./extracted_model")

# List the extracted files
extracted_files = os.listdir("./extracted_model")
print("Extracted files:", extracted_files)

Extracted files: ['model_d_finetuned']


In [4]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# Load the model and tokenizer from the extracted directory
model_path = "./extracted_model/model_d_finetuned"
model = AutoPeftModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Verify the model is loaded
print("Model and tokenizer loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


In [7]:
def translate_german_to_french(german_sentence):
    prompt = f"""### Instruction:
Translate the following German sentence to French.
### Input:
{german_sentence.strip()}
### Translation:"""
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {key: value.to("cuda") for key, value in inputs.items()}
    outputs = model.generate(**inputs, max_length=100)
    french_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translation = french_text.split("### Translation:")[-1].strip()
    translation = translation.split("\n")[0].strip()
    return translation

In [13]:
# Create the Gradio interface with customizations
iface = gr.Interface(
    fn=translate_german_to_french,
    inputs=gr.Textbox(label="Enter a German sentence", lines=3, placeholder="Type German text here..."),
    outputs=gr.Textbox(label="French Translation"),
    title="German to French Translation",
    description="Enter a German sentence and get the French translation using a fine-tuned model.",
)


iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab6678172f0cb12951.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
